### Make a python web based application where you'll enter the text and it'll browse all the file system from each directory and merge them .

In [15]:
import os
import string
import tkinter as tk
from tkinter.messagebox import showinfo

def drives():
    """
    This function returns a list of drives available in a system.
    """
    try:
        drive = string.ascii_uppercase #Contrains all the characterfrom a-z 
        valid_drivers = []
        for each_drive in drive:
            if os.path.exists(each_drive+":\\"):    # If any drive exists then it'll append to valid_drivers
                valid_drivers.append(each_drive+":\\")
    except Exception as e:
        print('The error encounterd in driver function is : ', e)
    finally:
        return valid_drivers
    
def search_file(filename):
    """
    This function searches for a file name in all the drivers of a computer.
    Argument to be passed in string, which is a part of file name or the file
    extension. 
    eg. Passing 'test' as an argument will return all the files with 'test'
        in the name. 
        Passing '*.docx' will return all the files with docx extension.
    
    This is a generator function and returns the result set having the directory 
    and file as a generator.
    """
    # default search for extension as false
    search_ext = False     # Assuming search not based on extension
    # Converting to lower case
    filename = filename.casefold()
    
    # Search will be done based on extension if the user has entered *.ext
    # where ext can be extension like txt, docx, mp3, avi, xlsx etc.
    if filename.startswith('*.'):                     # Checking if user searching based on extension
        extension = filename.replace('*.', '')       #Replacing the character before . to get only extension
        search_ext = True
        
    # Get all the drivers present on the desktop into list
    all_drives = drives()                            # Getting all the list of drives 
    
    for drive in all_drives:
        for path, dirs, files in os.walk(drive):    # walk will return all the files and paths to those files and directories
            for file in files:
                # Checking if the file content is present in list of files that if user is searching based on extension, if not then we will search based on file name
                if (search_ext and file.casefold().endswith(extension)) or ((not search_ext) and filename in file.casefold()):    
                    yield(path+'\\'+file) # Yield works as a generator instead of storing file to a variable which increase burden 
                                      # of the memory it keep on throwing file names
    
    
def merge_files(search_results, ext):
    """
    This function will search the file with supplied extension fro the search
    results. It will write the contents to a new file 'merged file' in the 
    current working directory 
    
    Arguments -
        search result - a generator function with the search results
        ext - extension of the files to be merged. It supports the below 
        mentioned extensions
            txt
    """
    lst = []
    f = open('merged_file.txt', 'w')
    for i in search_results:
        if i.casefold().endswith('.'+ext):
            src = open(i,'r')
            f.write(src.read())
            src.close()
            lst.append(i)
    f.close()
    return lst
            
     
    
def validate_contents():
    """
    This function checks if user has entered data on the search 
    window. If the user has entered the data then call search_file
    to search for file in all the drivers.
    
    The search results are displayed o the search window.
    """
    search = searchtext.get()     # Accepting the user input to a variable
    T.configure(state='normal')   # Unprotecting output window
    T.delete('1.0', tk.END)       # Clearing out the result
    T.configure(state='disabled') # Protecting again
    if search.strip() != '':      # if input is not empty
        search = search_file(search.strip())        # Calling search_file()
        num= 0
        for i in search:
            T.configure(state='normal')             # Unprotecting the file
            T.insert(tk.END, i+'\n')                # Insering the redult
            T.configure(state='disabled')           # Protecting it again
            num+=1
        showinfo("Search results", "Total number of files found : "+str(num))
    else:
        showinfo('Search results', "Enter a file to be searched")    # If input is empty
        
def validate_merge():
    """
    This function checks if user has entered data on the search window.
    If user has entered the data then call merge_files to search for file 
    in all the drives. All the .txt files will be merged to 'merged.txt'
    
    The merged source files will be displayed in popup window.
    """
    search = searchtext.get()
    if search.strip() != '':
        files = ''
        files_merged = merge_files(search_file(search.strip()), 'txt')
        for i in files_merged:
            files += i + '\n'
        if files == '':
            files = 'No .txt files were found for merging'
        showinfo("The below files were merged to 'nerge_files.txt'", files)
    else:
        showinfo("Merge results", " Enter the file name to be searched and merged")
    
            
window = tk.Tk()     # To create UI 
# To name the title of the window
window.title('Desktop Search')

# Label the search entry field
tk.Label(window, text='Enter file to be searched').grid(row=0) # Position of the fild
searchtext = tk.Entry(window, width=150)   # size
searchtext.grid(row=0, column=1)

tk.Label(window, text='Search results---->').grid(row=1)    # Search result output

tk.Button(window, text='Search', command=validate_contents).grid(row=0, column=2)  # Search button

T = tk.Text(window, height=25, width= 150)
T.grid(row=1, column=1)
T.configure(state='disabled')   # To disable the output window for text entry

tk.Button(window, text="Merge .txt files from search results", fg='blue', command=validate_merge).grid(row=2, column=1)
tk.Button(window, text="QUIT", fg='red', command=window.destroy).grid(row=3, column=2)

tk.mainloop()   # To trigger the command we need to call mainloop